<a href="https://colab.research.google.com/github/jlinehan31/BDS/blob/master/Copy_of_lvt_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install df2gspread

In [ ]:
import pandas as pd 
from datetime import datetime as dt
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [ ]:
filename = '2022.05 - LVT Customer Data'
df = pd.read_csv(f'{filename}.csv')

In [ ]:
# Isolate date columns to prepare transformation
id_cols = df.columns[:7]
date_cols = df.columns[7:]

In [ ]:
# Transform data to long format
df_final = pd.melt(df, id_vars=id_cols, value_vars=date_cols, var_name='Date', value_name='MRR')

In [ ]:
# Convert `Date` column to date type
df_final['Date'] = pd.to_datetime(df_final['Date'])

In [ ]:
# Change null values in `MRR` to 0
df_final['MRR'] = df_final['MRR'].fillna(0)

In [ ]:
# Sort `df_final` by `Acumatica ID` and `Date` to prepare for export
df_final = df_final.sort_values(by=['Acumatica ID', 'Date']).reset_index(drop=True)

In [ ]:
# Calculate max date in `df_final`
max_date = df_final['Date'].max().strftime('%b-%y')

In [ ]:
# Export data to csv
# df_final.to_csv(f'MRR Data ({max_date}).csv', index=False)

In [ ]:
# Create GS scope and credentials for API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

google_key_file = 'lvt-customer-data-eaa28d6f0bfa.json' 
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1xEZVbXia6oE3eAQhw1agc4ifbE_m1-YZKM4EKGS5foE' #LVT Customer Data

In [ ]:
# Upload to Google Sheets `lvt_data_test`
d2g.upload(df_final, spreadsheet_key, 'Sheet1', credentials=credentials, row_names=False)

<Worksheet 'Sheet1' id:0>